In [ ]:
pip install tensorflow soundfile numpy scipy soundfile numpy tensorflow-hub

In [7]:
pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py39-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached async_timeout-5.0.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached frozenlist-1.5.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
  Using cached multidict-6.1.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.0 kB)
Using cached datasets-3.1.0-py3-none-any.whl (480 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached fsspec-2024.9.0-py3-none-any.whl (179 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [39]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import csv
import io
from scipy import signal
import soundfile as sf

model = hub.load('https://tfhub.dev/google/yamnet/1')

def load_audio(filename, target_sample_rate=16000):
    wav_data, sample_rate = sf.read(filename, dtype='float32')
    if wav_data.ndim > 1:
        wav_data = np.mean(wav_data, axis=1)
    if sample_rate != target_sample_rate:
        wav_data = signal.resample(wav_data, int(len(wav_data) * target_sample_rate / sample_rate))
    return wav_data

def class_names_from_csv(class_map_csv_text):
    class_map_csv = io.StringIO(class_map_csv_text)
    class_names = [display_name for (class_index, mid, display_name) in csv.reader(class_map_csv)]
    return class_names[1:]

def predict_labels(filename):
    wav_data = load_audio(filename)
    scores, embeddings, spectrogram = model(wav_data)
    
    class_map_path = model.class_map_path().numpy().decode('utf-8')
    class_map_csv_text = tf.io.read_file(class_map_path).numpy().decode('utf-8')
    class_names = class_names_from_csv(class_map_csv_text)

    mean_scores = tf.reduce_mean(scores, axis=0).numpy()
    top5_indices = np.argsort(mean_scores)[::-1][:5]
    top5_labels = [class_names[i] for i in top5_indices]
    return top5_labels

filename = "prev.mp3"
labels = predict_labels(filename)
print("Предсказанные метки:", labels)


ModuleNotFoundError: No module named 'tensorflow_hub'

In [11]:
def generate_description(labels):
    description = "Звук с характеристиками: " + ", ".join(labels)
    return description

description = generate_description(labels)
print("Описание:", description)


NameError: name 'labels' is not defined

In [ ]:
pip install transformers librosa torch ipywidgets torchvision


In [1]:
import json
import librosa
import torch
from transformers import AutoModel, AutoProcessor

processor = AutoProcessor.from_pretrained("laion/clap-htsat-unfused")
model = AutoModel.from_pretrained("laion/clap-htsat-unfused")

def load_audio(filename, target_sr=48000):
    audio, sr = librosa.load(filename, sr=target_sr)
    return audio, sr

def get_best_description(audio_file, descriptions):
    audio, sr = load_audio(audio_file)
    audio_inputs = processor(audios=audio, sampling_rate=sr, return_tensors="pt", padding=True)
    with torch.no_grad():
        audio_features = model.get_audio_features(**audio_inputs)
    text_inputs = processor(text=descriptions, return_tensors="pt", padding=True)
    with torch.no_grad():
        text_features = model.get_text_features(**text_inputs)
    similarities = torch.nn.functional.cosine_similarity(audio_features, text_features)
    best_match_index = similarities.argmax().item()
    best_match_description = descriptions[best_match_index]
    return best_match_description

audio_file = "prev.mp3"
# audio_file = "6036a3dfb3944_c.mp3"
# audio_file = "62e030c5cf5ca_c.mp3"
with open('descriptions.json', 'r') as file:
    descriptions = json.load(file)

best_description = get_best_description(audio_file, descriptions)
print("Лучшее описание для данного аудио:", best_description)


2024-12-02 11:44:06.391982: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-02 11:44:06.590080: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733129046.663777  945392 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733129046.685214  945392 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 11:44:06.877638: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Лучшее описание для данного аудио: buzzy, distorted bass with a heavy punch, grounding intense tracks


In [ ]:
import os
import json
import librosa
import torch
from transformers import AutoModel, AutoProcessor

processor = AutoProcessor.from_pretrained("laion/clap-htsat-unfused")
model = AutoModel.from_pretrained("laion/clap-htsat-unfused")

def load_audio(filename, target_sr=48000):
    audio, sr = librosa.load(filename, sr=target_sr)
    return audio, sr

def get_best_description(audio_file, descriptions):
    audio, sr = load_audio(audio_file)
    audio_inputs = processor(audios=audio, sampling_rate=sr, return_tensors="pt", padding=True)
    with torch.no_grad():
        audio_features = model.get_audio_features(**audio_inputs)
    text_inputs = processor(text=descriptions, return_tensors="pt", padding=True)
    with torch.no_grad():
        text_features = model.get_text_features(**text_inputs)
    similarities = torch.nn.functional.cosine_similarity(audio_features, text_features)
    best_match_index = similarities.argmax().item()
    best_match_description = descriptions[best_match_index]
    return best_match_description

with open('descriptions.json', 'r') as file:
    descriptions = json.load(file)

def process_folders(data_dir, labeled_dir, n, k):
    os.makedirs(labeled_dir, exist_ok=True)

    for folder_num in range(n, k + 1):
        folder_name = f"{folder_num:04d}"
        folder_path = os.path.join(data_dir, folder_name)

        if not os.path.isdir(folder_path):
            print(f"Папка {folder_path} не существует, пропускаем.")
            continue

        params_path = os.path.join(folder_path, "params.json")
        audio_file = os.path.join(folder_path, "prev.mp3")
        
        vital_file = None
        for file_name in os.listdir(folder_path):
            if file_name.endswith(".vital"):
                vital_file = os.path.join(folder_path, file_name)
                break

        if not os.path.isfile(params_path) or not os.path.isfile(audio_file) or not vital_file:
            print(f"Отсутствует params.json, prev.mp3 или .vital файл в {folder_path}, пропускаем.")
            continue

        with open(params_path, 'r') as file:
            params = json.load(file)

        try:
            with open(vital_file, 'r') as file:
                vital_content = file.read().strip()
                if not vital_content:
                    print(f"Файл {vital_file} пуст, пропускаем.")
                    continue
                vital_data = json.loads(vital_content)
        except json.JSONDecodeError:
            print(f"Файл {vital_file} не является валидным JSON, пропускаем.")
            continue

        try:
            best_description = get_best_description(audio_file, descriptions)
            params["description"] = best_description
            params["preset"] = vital_data

            output_file = os.path.join(labeled_dir, f"{folder_num}.json")
            with open(output_file, 'w') as file:
                json.dump(params, file, indent=4)

            print(f"Создан {output_file}")
        except Exception as e:
            print(f"Ошибка при обработке {folder_path}: {e}")

data_directory = "../data"
labeled_directory = "labeled_data"
n = 1
k = 10
process_folders(data_directory, labeled_directory, n, k)


Создан labeled_data/1.json
Создан labeled_data/2.json
Создан labeled_data/3.json
Создан labeled_data/4.json
Создан labeled_data/5.json
Создан labeled_data/6.json
Создан labeled_data/7.json
Создан labeled_data/8.json
Создан labeled_data/9.json
Создан labeled_data/10.json


In [15]:
import os
import json
import librosa
from datasets import load_dataset
from transformers import ClapModel, ClapProcessor

librispeech_dummy = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
audio_sample = librispeech_dummy[0]

model = ClapModel.from_pretrained("laion/larger_clap_music")
processor = ClapProcessor.from_pretrained("laion/larger_clap_music")

def load_audio(filename, target_sr=48000):
    audio, sr = librosa.load(filename, sr=target_sr)
    return audio, sr

def get_best_description(audio_file, descriptions):
    audio, sr = load_audio(audio_file)
    audio_inputs = processor(audios=audio, sampling_rate=sr, return_tensors="pt", padding=True)
    with torch.no_grad():
        audio_features = model.get_audio_features(**audio_inputs)
    text_inputs = processor(text=descriptions, return_tensors="pt", padding=True)
    with torch.no_grad():
        text_features = model.get_text_features(**text_inputs)
    similarities = torch.nn.functional.cosine_similarity(audio_features, text_features)
    best_match_index = similarities.argmax().item()
    best_match_description = descriptions[best_match_index]
    return best_match_description

with open('descriptions.json', 'r') as file:
    descriptions = json.load(file)

def process_folders(data_dir, labeled_dir, n, k):
    os.makedirs(labeled_dir, exist_ok=True)

    for folder_num in range(n, k + 1):
        folder_name = f"{folder_num:04d}"
        folder_path = os.path.join(data_dir, folder_name)

        if not os.path.isdir(folder_path):
            print(f"Папка {folder_path} не существует, пропускаем.")
            continue

        params_path = os.path.join(folder_path, "params.json")
        audio_file = os.path.join(folder_path, "prev.mp3")

        vital_file = None
        for file_name in os.listdir(folder_path):
            if file_name.endswith(".vital"):
                vital_file = os.path.join(folder_path, file_name)
                break

        if not os.path.isfile(params_path) or not os.path.isfile(audio_file) or not vital_file:
            print(f"Отсутствует params.json, prev.mp3 или .vital файл в {folder_path}, пропускаем.")
            continue

        with open(params_path, 'r') as file:
            params = json.load(file)

        try:
            with open(vital_file, 'r') as file:
                vital_content = file.read().strip()
                if not vital_content:
                    print(f"Файл {vital_file} пуст, пропускаем.")
                    continue
                vital_data = json.loads(vital_content)
        except json.JSONDecodeError:
            print(f"Файл {vital_file} не является валидным JSON, пропускаем.")
            continue
        
        try:
            best_description = get_best_description(audio_file, descriptions)
            params["description"] = best_description
            params["preset"] = vital_data

            output_file = os.path.join(labeled_dir, f"{folder_num}.json")
            with open(output_file, 'w') as file:
                json.dump(params, file, indent=4)

            print(f"Создан {output_file}")
        except Exception as e:
            print(f"Ошибка при обработке {folder_path}: {e}")

data_directory = "../data"
labeled_directory = "labeled_data_music"
n = 1
k = 500
process_folders(data_directory, labeled_directory, n, k)


Создан labeled_data_music/1.json
Создан labeled_data_music/2.json
Создан labeled_data_music/3.json
Создан labeled_data_music/4.json
Создан labeled_data_music/5.json
Создан labeled_data_music/6.json
Создан labeled_data_music/7.json
Создан labeled_data_music/8.json
Создан labeled_data_music/9.json
Создан labeled_data_music/10.json
Создан labeled_data_music/11.json
Создан labeled_data_music/12.json
Создан labeled_data_music/13.json
Создан labeled_data_music/14.json
Файл ../data/0015/Snare_Generator.vital пуст, пропускаем.
Создан labeled_data_music/16.json
Создан labeled_data_music/17.json
Создан labeled_data_music/18.json
Создан labeled_data_music/19.json
Создан labeled_data_music/20.json
Создан labeled_data_music/21.json
Создан labeled_data_music/22.json
Создан labeled_data_music/23.json
Создан labeled_data_music/24.json
Создан labeled_data_music/25.json
Создан labeled_data_music/26.json
Создан labeled_data_music/27.json
Создан labeled_data_music/28.json
Создан labeled_data_music/29.json

KeyboardInterrupt: 

In [ ]:
import os
import json
import librosa
from datasets import load_dataset
from transformers import ClapModel, ClapProcessor

librispeech_dummy = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
audio_sample = librispeech_dummy[0]

model = ClapModel.from_pretrained("laion/larger_clap_general")
processor = ClapProcessor.from_pretrained("laion/larger_clap_general")

def load_audio(filename, target_sr=48000):
    audio, sr = librosa.load(filename, sr=target_sr)
    return audio, sr

def get_best_description(audio_file, descriptions):
    audio, sr = load_audio(audio_file)
    audio_inputs = processor(audios=audio, sampling_rate=sr, return_tensors="pt", padding=True)
    with torch.no_grad():
        audio_features = model.get_audio_features(**audio_inputs)
    text_inputs = processor(text=descriptions, return_tensors="pt", padding=True)
    with torch.no_grad():
        text_features = model.get_text_features(**text_inputs)
    similarities = torch.nn.functional.cosine_similarity(audio_features, text_features)
    best_match_index = similarities.argmax().item()
    best_match_description = descriptions[best_match_index]
    return best_match_description

with open('descriptions.json', 'r') as file:
    descriptions = json.load(file)

def process_folders(data_dir, labeled_dir, n, k):
    os.makedirs(labeled_dir, exist_ok=True)

    for folder_num in range(n, k + 1):
        folder_name = f"{folder_num:04d}"
        folder_path = os.path.join(data_dir, folder_name)

        if not os.path.isdir(folder_path):
            print(f"Папка {folder_path} не существует, пропускаем.")
            continue

        params_path = os.path.join(folder_path, "params.json")
        audio_file = os.path.join(folder_path, "prev.mp3")

        vital_file = None
        for file_name in os.listdir(folder_path):
            if file_name.endswith(".vital"):
                vital_file = os.path.join(folder_path, file_name)
                break

        if not os.path.isfile(params_path) or not os.path.isfile(audio_file) or not vital_file:
            print(f"Отсутствует params.json, prev.mp3 или .vital файл в {folder_path}, пропускаем.")
            continue

        with open(params_path, 'r') as file:
            params = json.load(file)

        try:
            with open(vital_file, 'r') as file:
                vital_content = file.read().strip()
                if not vital_content:
                    print(f"Файл {vital_file} пуст, пропускаем.")
                    continue
                vital_data = json.loads(vital_content)
        except json.JSONDecodeError:
            print(f"Файл {vital_file} не является валидным JSON, пропускаем.")
            continue

        try:
            best_description = get_best_description(audio_file, descriptions)
            params["description"] = best_description
            params["preset"] = vital_data

            output_file = os.path.join(labeled_dir, f"{folder_num}.json")
            with open(output_file, 'w') as file:
                json.dump(params, file, indent=4)

            print(f"Создан {output_file}")
        except Exception as e:
            print(f"Ошибка при обработке {folder_path}: {e}")

data_directory = "../data"
labeled_directory = "labeled_data_general"
n = 1
k = 10
process_folders(data_directory, labeled_directory, n, k)


Создан labeled_data_general/1.json
Создан labeled_data_general/2.json
Создан labeled_data_general/3.json
Создан labeled_data_general/4.json
Создан labeled_data_general/5.json
Создан labeled_data_general/6.json
Создан labeled_data_general/7.json
Создан labeled_data_general/8.json
Создан labeled_data_general/9.json
Создан labeled_data_general/10.json
